In [1]:
from util import *
import os
from glob import glob
from tqdm.contrib.concurrent import process_map
import rapidfuzz # Fuzzy string matching

/usr/local/lib/python3.10/dist-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/home/nyou045/git/CliffErosion_Projections/util.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration

In [2]:
df = pd.read_excel("AOI_SLR_rates.xlsx")
df

,Taranaki,AOI,SSP 4.5 (p50),SSP 4.5 (p83),SSP 8.5 (p50),SSP 8.5 (p83),Rate SSP 4.5 (p50),Rate SSP 4.5 (p83),Rate SSP 8.5 (p50),Rate SSP 8.5 (p83)
0,NORTH,Mohakatino_River,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110
1,NORTH,NewPlymouth,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110
2,NORTH,NewPlymouth_Airport,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110
3,NORTH,Oakura,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110
4,NORTH,Oakura_South,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110
5,NORTH,Onaero,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110
6,NORTH,PariokariwaPoint,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110
7,NORTH,TongaporutuRiver,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110
8,NORTH,UrenuiRiver_North,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110
9,NORTH,Waitara,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110


In [3]:
file_AOIs = [f.replace("_intersects.shp", "") for f in os.listdir("Shapefiles") if f.endswith("_intersects.shp")]
sorted(file_AOIs)

['CapeEgmont',
 'HangatahuRiver_South',
 'Hawera_WaihiBeach',
 'Kakaramea',
 'KaupokonuiBeach',
 'ManaBayCliffs',
 'Manutahi',
 'MohakatinoRiver',
 'NewPlymouthAirport',
 'NewPlymouthCliffs',
 'Oakura',
 'OakuraSouth',
 'Oeo',
 'OhaweBeach',
 'OnaeroCliff',
 'OpunakeBeachCliffs',
 'PariokariwaPointCliffs',
 'Pihama',
 'Rahotu',
 'TongapurutuRiverCliffs',
 'UrenuiRiverNorthCliffs',
 'WainuiBeach',
 'WaipipiBeachCliffs',
 'Waitara']

In [4]:
# When fuzzy matching, ignore these strings
strings_to_delete = ["_", "Cliffs", "Cliff"]

def fuzz_preprocess(filename):
    for s in strings_to_delete:
        filename = filename.replace(s, "")
    # Case-insensitive
    filename = filename.lower()
    # Ignore extension
    filename = os.path.splitext(filename)[0]
    # Basename only
    filename = os.path.basename(filename)
    return filename

def get_match(filename):
    match, score, index = rapidfuzz.process.extractOne(query=filename, choices=file_AOIs, processor=fuzz_preprocess)
    return match, score

df["match"], df["match_score"] = zip(*df.AOI.apply(get_match))
print("Perfect matches:", sum(df.match_score == 100))
print("Imperfect matches:", sum(df.match_score < 100))
df.sort_values(by="match_score")

Perfect matches: 22
Imperfect matches: 2


,Taranaki,AOI,SSP 4.5 (p50),SSP 4.5 (p83),SSP 8.5 (p50),SSP 8.5 (p83),Rate SSP 4.5 (p50),Rate SSP 4.5 (p83),Rate SSP 8.5 (p50),Rate SSP 8.5 (p83),match,match_score
7,NORTH,TongaporutuRiver,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,TongapurutuRiverCliffs,93.750000
11,SOUTH,HangatahuaRiver_South,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,HangatahuRiver_South,97.435897
21,SOUTH,Rahotu,0.58,0.78,0.84,1.10,0.0058,0.0078,0.0084,0.0110,Rahotu,100.000000
20,SOUTH,Pihama,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,Pihama,100.000000
19,SOUTH,OpunakeBeach,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,OpunakeBeachCliffs,100.000000
18,SOUTH,OhaweBeach,0.57,0.78,0.83,1.10,0.0057,0.0078,0.0083,0.0110,OhaweBeach,100.000000
17,SOUTH,Oeo,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,Oeo,100.000000
16,SOUTH,Manutahi,0.57,0.78,0.83,1.10,0.0057,0.0078,0.0083,0.0110,Manutahi,100.000000
15,SOUTH,ManaBay,0.57,0.78,0.83,1.10,0.0057,0.0078,0.0083,0.0110,ManaBayCliffs,100.000000
14,SOUTH,KaupokonuiBeach,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,KaupokonuiBeach,100.000000


In [5]:
SLR_rate_column_names = df.columns[df.columns.str.startswith("Rate SSP")]
SLR_rate_column_names

Index(['Rate SSP 4.5 (p50)', 'Rate SSP 4.5 (p83)', 'Rate SSP 8.5 (p50)',
       'Rate SSP 8.5 (p83)'],
      dtype='object')

In [6]:
# Run all models on all AOIs in parallel
def process_file(index_and_row):
    i, row = index_and_row
    site = row.match
    gdf = gpd.read_file(f"Shapefiles/{site}_intersects.shp")
    gdf.crs = 2193
    gdf = enrich_df(gdf)
    azimuth_lookup = get_azimuth_dict(f"Shapefiles/{site}_TransectLines.shp")
    for SLR_rate_column_name in SLR_rate_column_names:
        results = predict(gdf, azimuth_lookup, Proj_SLR=row[SLR_rate_column_name])
        for model in SUPPORTED_MODELS:
            results.set_geometry(f"{model}_model_point", inplace=True, crs=2193)
            polygon = prediction_results_to_polygon(results)
            output_shapefile = f"Projected_Shoreline_Polygons/{site}_{model}_{SLR_rate_column_name}.shp"
            polygon.to_file(output_shapefile, driver="ESRI Shapefile")

_ = process_map(process_file, df.iterrows())

0it [00:00, ?it/s]

In [7]:
# Compare different SLR rates
site = df.match.sample(1).iloc[0]
print(site)
gdf = gpd.read_file(f"Shapefiles/{site}_intersects.shp")
gdf.crs = 2193
gdf = enrich_df(gdf)
azimuth_lookup = get_azimuth_dict(f"Shapefiles/{site}_TransectLines.shp")
low_proj_slr = predict(gdf, azimuth_lookup, Proj_SLR=0.007)
high_proj_slr = predict(gdf, azimuth_lookup, Proj_SLR=0.015)
m = gpd.GeoSeries(low_proj_slr.sqrt_model_point, crs=2193).explore(color="blue", tiles="Esri.WorldImagery")
gpd.GeoSeries(high_proj_slr.sqrt_model_point, crs=2193).explore(color="green", m=m)

NewPlymouthAirport
